In [0]:
SELECT * FROM gizmobox.bronze.v_customers where customer_id IS NOT NULL
ORDER BY customer_id

In [0]:
SELECT DISTINCT * FROM gizmobox.bronze.v_customers where customer_id IS NOT NULL
ORDER BY customer_id

In [0]:
SELECT customer_id, MAX(created_timestamp)
FROM gizmobox.bronze.v_customers 
WHERE customer_id IS NOT NULL 
GROUP BY customer_id
ORDER BY customer_id

In [0]:
CREATE OR REPLACE TEMPORARY VIEW v_customers_distinct AS
SELECT DISTINCT * FROM gizmobox.bronze.v_customers 
WHERE customer_id IS NOT NULL ORDER BY customer_id

In [0]:
SELECT customer_id, MAX(created_timestamp) as max_created_timestamp
FROM v_customers_distinct GROUP BY customer_id

In [0]:
WITH cte_max AS (
  SELECT customer_id, MAX(created_timestamp) as max_created_timestamp
  FROM v_customers_distinct GROUP BY customer_id
)
SELECT CAST(t.created_timestamp AS TIMESTAMP) as created_timestamp,
t.customer_id, t.customer_name, CAST(t.date_of_birth AS DATE) AS date_of_birth,
t.email, CAST(t.member_since AS DATE) as member_since, t.telephone
FROM v_customers_distinct t
JOIN cte_max m ON t.customer_id = m.customer_id AND t.created_timestamp = m.max_created_timestamp

In [0]:
CREATE TABLE gizmobox.silver.customers
AS
WITH cte_max AS (
  SELECT customer_id, MAX(created_timestamp) as max_created_timestamp
  FROM v_customers_distinct GROUP BY customer_id
)
SELECT CAST(t.created_timestamp AS TIMESTAMP) as created_timestamp,
t.customer_id, t.customer_name, CAST(t.date_of_birth AS DATE) AS date_of_birth,
t.email, CAST(t.member_since AS DATE) as member_since, t.telephone
FROM v_customers_distinct t
JOIN cte_max m ON t.customer_id = m.customer_id AND t.created_timestamp = m.max_created_timestamp

In [0]:
SELECT * FROM gizmobox.silver.customers

In [0]:
SELECT payment_id, payment_timestamp, payment_status, payment_method FROM gizmobox.bronze.payments

In [0]:
SELECT 
  payment_id, 
  CAST(date_format(payment_timestamp, 'yyyy-MM-dd') AS DATE) as payment_date, 
  date_format(payment_timestamp, 'HH:mm:ss') as payment_time, 
  payment_status, 
  payment_method 
FROM gizmobox.bronze.payments

In [0]:
SELECT 
  payment_id, 
  CAST(date_format(payment_timestamp, 'yyyy-MM-dd') AS DATE) as payment_date, 
  date_format(payment_timestamp, 'HH:mm:ss') as payment_time, 
  CASE payment_status 
    WHEN 1 THEN 'Success' 
    WHEN 2 THEN 'Pending'
    WHEN 3 THEN 'Cancelled'
    WHEN 4 THEN 'FAILED'
    END AS payment_status, 
  payment_method 
FROM gizmobox.bronze.payments

In [0]:
CREATE TABLE gizmobox.silver.payments
AS
SELECT 
  payment_id, 
  CAST(date_format(payment_timestamp, 'yyyy-MM-dd') AS DATE) as payment_date, 
  date_format(payment_timestamp, 'HH:mm:ss') as payment_time, 
  CASE payment_status 
    WHEN 1 THEN 'Success' 
    WHEN 2 THEN 'Pending'
    WHEN 3 THEN 'Cancelled'
    WHEN 4 THEN 'FAILED'
    END AS payment_status, 
  payment_method 
FROM gizmobox.bronze.payments

In [0]:
SELECT * FROM gizmobox.silver.payments

In [0]:
SELECT 
  refund_id, 
  payment_id,
  refund_amount,
  refund_reason
FROM hive_metastore.bronze.refunds

In [0]:
SELECT 
  refund_id, 
  payment_id,
  refund_amount,
  split(refund_reason, ':')[0] AS refund_reason,
  split(refund_reason, ':')[1] AS refund_source
FROM hive_metastore.bronze.refunds

In [0]:
SELECT 
  refund_id, 
  payment_id,
  CAST(date_format(refund_timestamp, 'yyyy-MM-dd') AS DATE) AS refund_date,
  date_format(refund_timestamp, 'HH:mm:ss') AS refund_time,
  refund_amount,
  split(refund_reason, ':')[0] AS refund_reason,
  split(refund_reason, ':')[1] AS refund_source
FROM hive_metastore.bronze.refunds

In [0]:
CREATE SCHEMA hive_metastore.silver

In [0]:
CREATE TABLE hive_metastore.silver.refunds AS
SELECT 
  refund_id, 
  payment_id,
  CAST(date_format(refund_timestamp, 'yyyy-MM-dd') AS DATE) AS refund_date,
  date_format(refund_timestamp, 'HH:mm:ss') AS refund_time,
  refund_amount,
  split(refund_reason, ':')[0] AS refund_reason,
  split(refund_reason, ':')[1] AS refund_source
FROM hive_metastore.bronze.refunds

In [0]:
SELECT * FROM hive_metastore.silver.refunds

In [0]:
DESC EXTENDED hive_metastore.silver.refunds

In [0]:
SELECT * FROM gizmobox.bronze.v_memberships

In [0]:
SELECT 
regexp_extract(path, '.*/([0-9]+)\\.png$', 1) AS customer_id,
content as membership_card
FROM gizmobox.bronze.v_memberships

In [0]:
CREATE TABLE gizmobox.silver.membership_cards AS
SELECT 
regexp_extract(path, '.*/([0-9]+)\\.png$', 1) AS customer_id,
content as membership_card
FROM gizmobox.bronze.v_memberships

In [0]:
SELECT * FROM gizmobox.silver.membership_cards